# Road segmentation

In [ ]:
%matplotlib inline
import numpy as np
import os,sys
from helpers import *
from plots import *

## Parameters

In [ ]:
# Model
from sklearn import linear_model
model = linear_model.LogisticRegression(C=1e5, class_weight="balanced", solver="lbfgs")

# Number of training images
n = 50

# AIcrowd expects patches of 16 by 16 pixels.
patch_size = 16

# percentage of pixels > 1 required to assign a foreground label to a patch
foreground_threshold = 0.25

# Extraction function
extraction_func = extract_features_2d

# Create a submission file ?
submit = True

## Data extraction

In [ ]:
# Load a set of images
imgs, gt_imgs = load_training_images(n)

In [ ]:
# Extract patches from all images
img_patches = get_patches(imgs, patch_size)
gt_patches = get_patches(gt_imgs, patch_size)

In [ ]:
# Get features for each image patch
X, Y = get_data_from_patches(img_patches, gt_patches, foreground_threshold, extraction_func)

## Training and predictions

In [ ]:
# Training
model.fit(X, Y)

In [ ]:
# Predict on the training set
Z = model.predict(X)

TPR = true_positive_rate(Z, Y)
print('True positive rate = ' + str(TPR))

In [ ]:
# Predict on test set and create submission
if submit:
    create_submission(model, extraction_func, patch_size)

## Visualization of a single test image

In [ ]:
# Load a test image
img_idx = 1

img_path = "Datasets/test_set_images/test_{0}/test_{0}.png".format(img_idx)
img = load_image(img_path)

In [ ]:
# Run predictions
Xi = get_data_from_img(img, extraction_func, patch_size)
Zi = model.predict(Xi)

In [ ]:
if Xi.shape[1] == 2:
    show_data_points(Xi, Zi)

In [ ]:
# Display prediction as an image
w = img.shape[0]
h = img.shape[1]
predicted_img = label_to_img(w, h, patch_size, patch_size, Zi)
show_img_concatenated(img, predicted_img)

In [ ]:
show_img_overlay(img, predicted_img)